In [2]:
#imports
import numpy as np
import os
import pandas as pd
import json
import sklearn
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# import scikit-learn as sklearn

In [3]:
# load the datasets

def loop_slices(path, num_slices=20):
  cnt = 0
  mpd_playlists = []
  filenames = os.listdir(path)
  for fname in sorted(filenames):
    print(fname)
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
        cnt += 1
        fullpath = os.sep.join((path, fname))
        f = open(fullpath)
        js = f.read()
        f.close()
        current_slice = json.loads(js)      
        
        # Create a list of all playlists
        for playlist in current_slice['playlists']:
            mpd_playlists.append(playlist)      
    
        if cnt == num_slices:
            break  
    return mpd_playlists
# Path where the json files are extracted
million_playlists_json = 'data/spotify_million_playlist_dataset/data/'
playlists = loop_slices(million_playlists_json, num_slices=20)
#convert to a df
df = pd.json_normalize(playlists)

# idea for how to combine features https://medium.com/analytics-vidhya/music-recommender-system-part-2-ff4c3f54cba3

#load our spotify features dataset
spotify_features = pd.read_csv("data/SpotifyFeatures.csv")

mpd.slice.0-999.json


In [4]:
display(df)
df["tracks"].head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN


0    [{'pos': 0, 'artist_name': 'Missy Elliott', 't...
1    [{'pos': 0, 'artist_name': 'Survivor', 'track_...
2    [{'pos': 0, 'artist_name': 'Hoody', 'track_uri...
3    [{'pos': 0, 'artist_name': 'Camille Saint-Saën...
4    [{'pos': 0, 'artist_name': 'The Smashing Pumpk...
Name: tracks, dtype: object

In [5]:

display(spotify_features)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


In [4]:
# we want to use random forest classifier to predict attributes of songs 
# we will first try to use all song features to predict valence

spotify_features = pd.read_csv("data/SpotifyFeatures.csv")

# major -> 1, minor -> 0
spotify_features["mode"].replace(["Major","Minor"], [1,0], inplace=True)

keys = sorted(spotify_features['key'].unique())
print(keys)
spotify_features['key'].replace(keys, [1,2,3,4,5,6,7,8,9,10,11,12], inplace=True)

#display(spotify_features)

x = spotify_features[["popularity", "acousticness", "danceability", "duration_ms", "energy", "instrumentalness", "key", "liveness", "loudness","valence", "speechiness", "tempo"]]
y = spotify_features[["mode"]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, train_size=.8)

print(x.shape + y.shape)

print(x_train.shape + y_train.shape) 


['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
(232725, 12, 232725, 1)
(186180, 12, 186180, 1)


In [5]:

# leave random state as set for now

#rf = RandomForestRegressor(random_state=0)
rf = RandomForestClassifier(random_state=0)

rf.fit(x_train,y_train)

#acc = rf.score(x_test,y_test)


C:\Users\alexp\AppData\Local\Temp\ipykernel_73100\3397728318.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train,y_train)


Error: Canceled future for execute_request message before replies were done